In [4]:
from sklearn.datasets import fetch_openml
mnist = fetch_openml('mnist_784', data_home=".")

In [7]:
len(mnist.data)

70000

In [11]:
mnist.data[0].shape

(784,)

In [29]:
img_shape = (28,28)

In [41]:
from PIL import Image
import numpy as np
img = Image.fromarray(np.uint8(mnist.data[0].reshape(img_shape)), mode = 'P')
img

In [61]:
import math
import torch
import torch.nn as nn
import torch.nn.functional as F

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [62]:
import random
manualSeed = 42
random.seed(manualSeed)
torch.manual_seed(manualSeed)

In [131]:
img_channels = 1
latent_dim = 16
feature_map_size = 32
number_of_layers = 4

class Generator(nn.Module):
    """
    Mostly following the implementation of the DCGAN.
    """
    def __init__(self):
        super(Generator, self).__init__()
        
        layers = []
        

        out_channels = feature_map_size*int(math.pow(2, number_of_layers-1))
        
        layers.append(nn.ConvTranspose2d(in_channels=latent_dim, out_channels = out_channels, kernel_size = 4, stride = 1, padding = 0, bias=False))
        
        """
        Strides of 2 on the ConvTranspose2d with knernel_size of 4 and padding of 1 will double the resolution of the output at each layer.
        Each layer after the first double de resolution.
        
        Notice that BatchNorm2D and ReLU are applied after each ConvTranspose2D intended to be fed to another (so not the last one!)
        """
 
        for i_layer in reversed(range(number_of_layers-1)):
        
            layers.append(nn.BatchNorm2d(out_channels))
            layers.append(nn.ReLU(out_channels))

            in_channels = out_channels
            out_channels = int(out_channels/2)
            layers.append(
                nn.ConvTranspose2d(
                    in_channels=in_channels,
                    out_channels = out_channels if i_layer != 0 else img_channels, # The last layer should have the same number of channels than the generated image
                    kernel_size = 4,
                    stride = 2,
                    padding = 1,
                    bias=False))
            
        
        self.generator = nn.Sequential(
            *layers,
            nn.Tanh()
        )

    def forward(self, z):
        return self.generator(z)

In [132]:
gen = Generator().to(device)

In [133]:
gen

Generator(
  (generator): Sequential(
    (0): ConvTranspose2d(16, 256, kernel_size=(4, 4), stride=(1, 1), bias=False)
    (1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): ConvTranspose2d(256, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (4): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace=True)
    (6): ConvTranspose2d(128, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (7): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (8): ReLU(inplace=True)
    (9): ConvTranspose2d(64, 1, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (10): Tanh()
  )
)

In [134]:
noise = torch.randn(1, latent_dim, 1, 1, device=device)

gen(noise).shape

torch.Size([1, 1, 32, 32])